<a href="https://colab.research.google.com/github/patriciacs99/WeaponDetectionYOLOv5/blob/main/WeaponDetectionYOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔫 Detección de armas de fuego con YOLOV5 
*Autor: Patricia Corral Sanz*

> En este proyecto se han utilizado los siguientes snipets para detectar armas de fuego y clasificarlas en armas largas (escopetas, fusiles, ametralladoras y francotiradores) y cortas (pistolas y revólveres).  
Para los experimentos realizados se ejecutarán las celdas necesarias para cada uno de ellos en función del objetivo del mismo. En cada caso cambiando los valores necesarios tal y como se indica en este cuaderno.  
Se utilizarán distintos datasets que pueden encontrarse en:  
https://drive.google.com/drive/folders/15O3lpCT-JYyuhEc5WftPzr0vCELjLPSS?usp=drive_link  




---

In [ ]:
#importar librerías
import os
import shutil
import random
import cv2
import torch
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import glob
import re
import time
from pathlib import Path
import itertools
from IPython.display import Image,display
from google.colab.patches import cv2_imshow

In [ ]:
#Comprobar GPU asignada
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

##Preparar de los datos y Yolov5



> En este apartado accederemos a Google Drive, donde tendremos organizados los datasets a utilizar en los experimentos.   
Clonaremos el repositorio de YOLOv5 de Ultralytics para poder trabajar con él e instalaremos los requisitos del mismo.  
Descargaremos también los modelos de YOLOv5 con pesos pre-entrenados con el dataset de COCO que vayamos a utilizar. En nuestro caso, solo utilizaremos YOLOv5s y YOLOv5m.  

⚠ *Como resultado de la ejecución de los ficheros de YOLOv5 (train.py,detect.py y val.py) se generará en el directorio de YOLOv5 que hemos clonado una carpeta "runs" donde encontraremos los resultados de las ejecuciones.*



> En el fichero data.yaml se especifican las rutas a las imágenes de test, validación y entrenamiento de nuestro dataset. Por tanto, antes de ejecutar las celdas es necesario asegurarse de que el fichero contiene las rutas correctas. De lo contrario YOLOv5 no encontrará las imágenes con las que trabajar.  
* train: ../ruta_imagenes_entrenamiento
* val:   ../ruta_imagenes_validacion
* test:  ../ruta_imagenes_test












In [ ]:
#conectar con google drive

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


In [ ]:
#clonar YOLO V5 
%cd content
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
#Instalar YOLOv5
%cd /content/yolov5
%pip install -qr requirements.txt

In [ ]:
# Descargar modelo con pesos preentrenado
'''
  Modelos a probar:
    - yolov5s.pt
    - yolov5m.pt
    - yolov5l.pt
    - yolov5x.pt
'''
# Sustituir "yolov5m.pt" en el link por el nombre del modelo a descargar
!wget https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5m.pt

## 👟 Entrenar modelo


> Entrenaremos el modelo para nuestro caso en particular: detectar armas de fuego cortas y largas.  
En las siguientes celdas, deben sustituirse por sus correspondientes rutas: 
*   **ruta_fichero_configuracion**: ruta al fichero yaml del dataset utilizado.
*   **ruta_pesos_modelo_YOLOv5**: ruta al modelo con los pesos pre-entrenados previamente descargado. 
*   **ruta_directorio_resultados**: ruta al directorio donde se han generado los resultados.







In [ ]:
!python train.py --img 416 --batch 32 --epochs 100 --data ruta_fichero_configuracion --weights ruta_pesos_modelo_YOLOv5   --name "resultados_entrenamiento" --nosave 

In [ ]:
#Descargar resultados entrenamiento
from google.colab import files

!zip -r train_results_yolov5.zip ruta_directorio_resultados/resultados_entrenamiento
files.download('train_results_yolov5.zip')

### Resultados entrenamiento con Tensorboard


> Ejecutaremos Tensorboard para visualizar las gráficas resultado del entrenamiento.  
Sustituir:  
*   **directorio_entrenamiento**: ruta del diretorio donde se han generado los resultados del entrenamiento.









In [ ]:
%reload_ext tensorboard


In [ ]:
%tensorboard --logdir directorio_entrenamiento


## 🎯 Inferencia 

> Se usarán los pesos del modelo entrenado para detectar y clasificar armas en nuevas imágenes de test que no ha visto el modelo anteriormente.  En las siguientes celdas, deben sustituirse por sus correspondientes rutas:  
*   **ruta_imagenes_test**: ruta a las imagenes de test del dataset.
*   **ruta_pesos**: ruta a los pesos generados en el entrenamiento.
*   **ruta_detecciones**: ruta del directorio donde se han generados los resultados de la inferencia.









In [ ]:
!python detect.py --source ruta_imagenes_test --weights ruta_pesos --img 416  --save-txt #--save-crop

In [ ]:
#Mostrar las imagenes de test con las detecciones
for imageName in glob.glob("ruta_detecciones/*.jpg"):
  display(Image(filename=imageName))
  print("\n")

In [ ]:
#Descargar resultados detect
from google.colab import files

!zip -r train_detect_yolov5.zip ruta_detecciones
files.download('train_detect_yolov5.zip')

## 📈 Evaluacion


>   

> Extraeremos las métricas de el modelo sobre las nuevas imágenes de test para poder evaluar su desempeño.  
En las siguientes celdas, deben sustituirse por sus correspondientes rutas: 
*   **ruta_fichero_configuracion**: ruta al fichero yaml del dataset utilizado.
*   **ruta_pesos**: ruta donde se encuentra el fichero con los pesos resultado del entrenamiento del modelo.
*   **ruta_directorio_resultados**: ruta del directorio donde se han generado los resultados de la ejecución del fichero val.py.













In [ ]:
!python val.py --weights ruta_pesos --data ruta_fichero_configuracion --workers 0 --img 416 --save-txt --conf-thres 0.5 --task test

In [ ]:
#Descargar resultados entrenamiento
from google.colab import files

!zip -r train_metrics_yolov5.zip ruta_directorio_resultados
files.download('train_metrics_yolov5.zip')



---

## 📑 Códigos complementarios


> Los siguientes códigos han resultado útiles para la elaboración de los experimentos.



### Intersection Over Union


>Aun que YOLO calcula el mismo el IoU, aquí se calcula para cada imagen las intersecciones para poder dibujarlas en cada imagen.   
Al ejecutar las siguientes celdas, se calcula la intersección entre la etiqueta real y la detección, y se dibuja el Ground Truth sobre la imagen que contiene la detección para poder visualizar claramente la diferencia. 



In [ ]:
 #formato: x,y,w,h  ---> formato: x1,y1,x2,y2
def rectangulo(box):
    box_x1 = box[0] - box[2]/2
    box_y1 = box[1] - box[3]/2
    box_x2 = box[0] + box[2]/2
    box_y2 = box[1] + box[3]/2
    return [box_x1,box_y1,box_x2,box_y2]

In [ ]:
def intersection_over_union(target_bbox,predicted_bbox):
   
    #formato: x,y,w,h

    #Calcular esquinas bboxes
    tg = rectangulo(target_bbox)
    pred = rectangulo(predicted_bbox)

    #Calcular esquinas interseccion
    x1 = max(pred[0], tg[0])
    y1 = max(pred[1], tg[1])
    x2 = min(pred[2], tg[2])
    y2 = min(pred[3], tg[3])

    #area = w * h
    intersection = max(0,(x2 - x1)) * max(0,(y2 - y1))

    box1_area = abs((pred[2] - pred[0]) * (pred[3] - pred[1]))
    box2_area = abs((tg[2] - tg[0]) * (tg[3] - tg[1]))S

    return intersection / (box1_area + box2_area - intersection + 1e-16)


In [ ]:
#Pintar detecciones,IoU,groundTruth

predicted = "/content/yolov5/runs/detect/exp/labels/0205_jpg.rf.eb76fc2a71b61cec8cdf25fd3e51c6cb.txt"
real = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/Customized/WeaponsDataset.v8-weapons_strectch_oclussionsomited.yolov5pytorch/test/images/0205_jpg.rf.eb76fc2a71b61cec8cdf25fd3e51c6cb.jpg"

#Leer imagen deteccion
image = cv2.imread("/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/weapons.v2-weapons_blackedges_contrast.yolov5pytorch/test/images/0251_jpg.rf.1d75a35dbae43d7e1f13a8266a5d8726.jpg")

with open(real) as archivo: # Abrimos el fichero con las coordenadas del gt
    for linea in archivo:
      coord1 =[] # lista que almacena las coordenadas del fichero
      clase = linea[0] # sacamos la clase de la deteccion - primer caracter de la linea leida
      coord1.append(linea[1:].split()) # Eliminamos ese primer caracter y añadimos el resto a la lista 
      coord1 =list(itertools.chain(*coord1)) # Aplanamos la lista
      x=0
      for i in coord1: # Castear los valores de string a float
            coord1[x]= float(i)
            x +=1
      coords= [] # Habra que guardar las coordenadas de todas las posibles detecciones para compararlas con el gt
      if os.path.exists(predicted): # Si existe el archivo con las detecciones repetimos el proceso para cada deteccion
        with open(predicted) as archivo:
          for linea in archivo:
                coord2 =[]
                coord2.append(linea[1:].split())
                coord2= list(itertools.chain(*coord2))
                x=0
                for i in coord2:
                        coord2[x]= float(i)
                        x +=1
                coords.append(coord2)
        
        ious = [] # Almacenamos las ious
        for i in coords:
            ious.append(intersection_over_union(i,coord1))
        # Nos quedamos con la deteccion con mayor IoU
        mayor = ious.index(max(ious)) 
        deteccion = coords[mayor]
      
        pred = rectangulo(deteccion)
        gt = rectangulo(coord1)
        iou = max(ious)

      else:
        pred = rectangulo([0,0,0,0])
        gt = rectangulo(coord1)
        iou = 0.000

      for i in range(0,4):
          gt[i] = int(gt[i]*416)

      start_point = tuple(gt[:2])
      end_point = tuple(gt[2:])
      
      #Imprimir rectangulos sobre la imagen de la deteccion
      cv2.rectangle(image,start_point,end_point,(0, 255, 0), 4)
      
      cv2.putText(image, "IoU: {:.3f}".format(iou), (gt[0]-10, gt[1]-10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
      if iou !=0:
        cv2.putText(image, "Class: "+ clase, (gt[0]+105, gt[1]- 10),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 2)
        
# Mostrar la imagen con el IoU, la deteccion y el ground truth
cv2_imshow(image)


### Tamaño de las instancias de una imagen 


> Código para dividir las imágenes de test según su tamaño utilizado en el experimento 2.



In [ ]:
# Crear 3 directorios para dividir las imagenes
os.makedirs("dir_peq/images")
os.makedirs("dir_peq/labels")
os.makedirs("dir_med/images")
os.makedirs("dir_med/labels")
os.makedirs("dir_gra/images")
os.makedirs("dir_gra/labels")

In [ ]:
dir_origen = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/weapons.v2-weapons_blackedges_contrast.yolov5pytorch/test/images/"

# Directorio imagenes
imagenes = glob.glob("/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/weapons.v2-weapons_blackedges_contrast.yolov5pytorch/test/images/*.jpg")

dir_gra = "/content/dir_gra"
dir_med = "/content/dir_med"
dir_peq = "/content/dir_peq"


for img in imagenes:
  image = cv2.imread(img)
  # Tamaño de la imagen (CUIDADO ESTO ESTÁ EN PIXELES)
  w = image.shape[0]
  h = image.shape[1]
  image_size = w*h
  real = img.replace("/images/","/labels/",1)
  real = real[:-3] + "txt"
  img_name = re.sub(dir_origen,'',img)
  img_label = img_name[:-3] + "txt"
  print(img_label)
  print(img_name)
  with open(real) as archivo: # Abrimos el fichero con las coordenadas del gt
    lista = archivo.readlines()
    num = len(lista)
    if num == 1:
        linea = lista[0]
       
        coord1 =[] # lista que almacena las coordenadas del fichero
        clase = linea[0] # sacamos la clase de la instancia - primer caracter de la linea leida
        coord1.append(linea[1:].split()) # Eliminamos ese primer caracter y añadimos el resto a la lista 
        coord1 =list(itertools.chain(*coord1)) # Aplanamos la lista
        x=0
        for i in coord1: # Castear los valores de string a float
              coord1[x]= float(i)
              x +=1
        # Sacamos los valores de altura y anchura de las coordenadas extraidas
        w1 = coord1[2] * 416
        h1 = coord1[3] * 416
        # Calculamos el tamaño de la instancia con dichos valores
        instance_size = w1*h1

        # Calculamos el porcentaje que ocupa dicha instancia sobre la imagen
        percentage_size = instance_size * 100 / image_size
        print("La instancia ocupa el "+ str(percentage_size) +"% de la imagen")
      
        if 0 <= percentage_size <= 15: 
          shutil.copy(img, "dir_peq/images/"+ img_name)
          shutil.copy(real,"dir_peq/labels/"+ img_label)
          print("Instancia peque")
        elif 16 <= percentage_size <= 35:
          shutil.copy(img, "dir_med/images/"+  img_name)
          shutil.copy(real,"dir_med/labels/"+ img_label)
          print("Instancia mediana")
        else:
          shutil.copy(img, "dir_gra/images/"+  img_name)
          shutil.copy(real,"dir_gra/labels/"+ img_label)
          print("Instancia grande")
      
    


In [ ]:
# Descargar imágenes por tamaño
from google.colab import files

!zip -r imagenes_medianas.zip /content/dir_med
!zip -r imagenes_pequeñas.zip /content/dir_peq
!zip -r imagenes_grandes.zip /content/dir_gra
files.download('imagenes_medianas.zip')
files.download('imagenes_pequeñas.zip')
files.download('imagenes_grandes.zip')

### Modificar valor etiquetas

>Si hubiese algun problema porque las clases de los dataset estuviesen cambiadas, este fragmento de código modifica el valor de la clase en las etiquetas.  
Sustituir **ruta_directorio_etiquetas** por la ruta del diretorio que 
contiene las etiquetas.








In [ ]:
# Codigo para reemplazar la clase
import glob 
dir = glob.glob('ruta_directorio_etiquetas/*.txt')

for elem in dir:
 fich1 = open(elem,'r')
 for line in fich1:
      if(line[0] == "0"):
        data = line.replace("0","1",1)
        fich2 = open(elem,'w')
        fich2.write(data)
        fich2.close()
      else:
        data = line.replace("1","0",1)
        fich2 = open(elem,'w')
        fich2.write(data)
        fich2.close()
 fich1.close()

         



In [ ]:
# Descargar directorio
from google.colab import files

!zip -r labels.zip ruta_directorio_etiquetas
files.download('labels.zip')